In [209]:
import requests
import base64
import csv
import json
from datetime import datetime, date
import urlib.parse
from spotify_client import *
from config import *

ModuleNotFoundError: No module named 'urlib'

In [ ]:
def refresh_accesss_token():
    client_creds = f'{client_id}:{client_secret}'
    client_creds_b64 = base64.b64encode(client_creds.encode())

    refresh_token_header = {
        'Authorization' : f'Basic {client_creds_b64.decode()}'
    }

    # def refresh_playlist_token():
    refresh_url =  'https://accounts.spotify.com/api/token'
    refresh_params = {
        'grant_type': 'refresh_token',
        'refresh_token': 'AQA92cE5oL-fTQvG1_objGmNUVNQ_6AseJUqjsNC2ujDEFeMTOPxBpLqQ6Ct2tkmLIp79RT6deD3WjdPRYDlhnU2YgautBPNYw5aGMjYVD-y35lz5_n1L88yxG2UA-7nd58'
        
    }

    refresh_data = urlencode(refresh_params)



    r_token = requests.post(refresh_url, data=refresh_params, headers=refresh_token_header)

    refresh_response = r_token.json()

    access_token = refresh_response['access_token']

    return access_token

In [ ]:
# def get_week_num():
playlist_token = refresh_accesss_token()
csv_path = path
def get_week_num():
    my_date = datetime.date.today() 
    year, week_num, day_of_week = my_date.isocalendar()
    this_week = week_num
    return this_week

In [ ]:
def get_album_tracks(album_ids):
    track_ids = []
#     request_data = json.dumps(uris)
    for album_id in album_ids:
        query = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
        response = requests.get(
            query,
            headers = {
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {playlist_token}'
            })
        response_json = response.json()
#         print(response_json)
        tracks = response_json['items']
        for track in tracks:
            track_ids.append(track['uri'])
    return track_ids

In [ ]:
def math_probs(a, b, c, d):
    e = a+b
    f = c+d
    return({'e':e, 'f':f})

g = math_probs(1,2,3,4)
print(g['e'])

In [ ]:
# def search_for_albums(csv_path):
artists=[]
albums=[]
albums_not_found = {'artist': [], 'album': []}
album_ids = set()
#     week_num = get_week_num()
week_num = 14
output_path = os.path.join('..','..', 'data', 'test',f'albums_not_found_wk_{week_num}.csv')


#     read in weekly metaScrape csv 
with open(csv_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
#         filter for artists and albums from current week
        if int(row['week_num'])==week_num:
            artists.append(row['artist'])
            albums.append(row['album'])
#   initialize spotify client
spotify = SpotifyAPI(client_id, client_secret)
for al, ar in zip(albums, artists):
#   search for album ids 
    temp = spotify.search({"album":al, "artist":ar}, search_type="album")
    try:
        parse_album_ids = (temp["albums"]["items"][0]["id"])
#   create dicitonary for albums not found
    except:
        albums_not_found['artist'] = ar
        albums_not_found['album'] = al
#     album_ids.add(parse_album_ids)
# try:
with open(output_path, 'w') as csvfile:
    fieldnames = ['artist', 'album']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(albums_not_found.values())
# print(albums_not_found.values())

#             writer.writerow(data)
#     except IOError:
#         print("I/O error")
#         return(get_album_tracks(album_ids))



In [ ]:
def refresh_accesss_token():
    client_creds = f'{client_id}:{client_secret}'
    client_creds_b64 = base64.b64encode(client_creds.encode())

    refresh_token_header = {
        'Authorization' : f'Basic {client_creds_b64.decode()}'
    }

    # def refresh_playlist_token():
    refresh_url =  'https://accounts.spotify.com/api/token'
    refresh_params = {
        'grant_type': 'refresh_token',
        'refresh_token': 'AQA92cE5oL-fTQvG1_objGmNUVNQ_6AseJUqjsNC2ujDEFeMTOPxBpLqQ6Ct2tkmLIp79RT6deD3WjdPRYDlhnU2YgautBPNYw5aGMjYVD-y35lz5_n1L88yxG2UA-7nd58'
        
    }

    refresh_data = urlencode(refresh_params)



    r_token = requests.post(refresh_url, data=refresh_params, headers=refresh_token_header)

    refresh_response = r_token.json()

    access_token = refresh_response['access_token']

    return access_token

def add_tracks_to_playlist(playlist_id):
    track_ids = [track for track in search_for_albums(csv_path)]
    track_limit = 100 
    # using list comprehension 
    batched_tracks = [track_ids[i * track_limit:(i + 1) * track_limit] for i in range((len(track_ids) + track_limit - 1) // track_limit)]  
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'

    for batch in batched_tracks:

        request_data = json.dumps(batch)

        response = requests.post(
            url,
            data=request_data,
            headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {playlist_token}'
            })
        return(response.json())

def create_playlist():
#     week_num = get_week_num()
    week_num = 14
    request_body = json.dumps({
        'name': f'2021-week {week_num} scrape',
        'description': f'metacritic rated albums for the {week_num}th of the year',
        'public': True
    })
    url = f'https://api.spotify.com/v1/users/{spotify_user_id}/playlists'


    response = requests.post(
        url,
        data = request_body,
        headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {playlist_token}'

    })
    response_json = response.json()
#     print(response_json)
    playlist_id = (response_json['id'])

    return add_tracks_to_playlist(playlist_id)

In [42]:
import requests
import base64
import csv
import json
from datetime import datetime, date
from spotify_client import *
from config import *

def refresh_accesss_token():
    client_creds = f'{client_id}:{client_secret}'
    client_creds_b64 = base64.b64encode(client_creds.encode())

    refresh_token_header = {
        'Authorization' : f'Basic {client_creds_b64.decode()}'
    }

    # def refresh_playlist_token():
    refresh_url =  'https://accounts.spotify.com/api/token'
    refresh_params = {
        'grant_type': 'refresh_token',
        'refresh_token': 'AQA92cE5oL-fTQvG1_objGmNUVNQ_6AseJUqjsNC2ujDEFeMTOPxBpLqQ6Ct2tkmLIp79RT6deD3WjdPRYDlhnU2YgautBPNYw5aGMjYVD-y35lz5_n1L88yxG2UA-7nd58'
        
    }

    refresh_data = urlencode(refresh_params)



    r_token = requests.post(refresh_url, data=refresh_params, headers=refresh_token_header)

    refresh_response = r_token.json()

    access_token = refresh_response['access_token']

    return access_token


# def get_week_num():
playlist_token = refresh_accesss_token()
csv_path = path
def get_week_num():
    my_date = datetime.date.today() 
    year, week_num, day_of_week = my_date.isocalendar()
    this_week = week_num
    return this_week


def get_album_tracks(album_ids):
    track_ids = []
#     request_data = json.dumps(uris)
    for album_id in album_ids:
        query = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
        response = requests.get(
            query,
            headers = {
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {playlist_token}'
            })
        response_json = response.json()
#         print(response_json)
        tracks = response_json['items']
        for track in tracks:
            track_ids.append(track['uri'])
    return track_ids

def search_for_albums(csv_path):
    artists=[]
    albums=[]
    albums_not_found = {'artist': [], 'album': []}
    album_ids = set()

#     read in weekly metaScrape csv 
    with open(csv_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
#         filter for artists and albums from current week
            if int(row['week_num'])==get_week_num():
                artists.append(row['artist'])
                albums.append(row['album'])
#   initialize spotify client
    spotify = SpotifyAPI(client_id, client_secret)
    for al, ar in zip(albums, artists):
#   search for album ids 
        temp = spotify.search({"album":al, "artist":ar}, search_type="album")
        try:
            parse_album_ids = (temp["albums"]["items"][0]["id"])
#   create dicitonary for albums not found
        except:
            albums_not_found['artist'].append(ar)
            albums_not_found['album'].append(al)
        album_ids.add(parse_album_ids)
    return(get_album_tracks(album_ids, ))


    
# def add_tracks_to_playlist(playlist_id):
#     get_track_features()
#     track_ids = [track for track in search_for_albums(csv_path)]
#     track_limit = 100 
#     # using list comprehension 
#     batched_tracks = [track_ids[i * track_limit:(i + 1) * track_limit] for i in range((len(track_ids) + track_limit - 1) // track_limit)]  
#     url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'

#     for batch in batched_tracks:

#         request_data = json.dumps(batch)

#         response = requests.post(
#             url,
#             data=request_data,
#             headers = {
#             'Content-Type': 'application/json',
#             'Authorization': f'Bearer {playlist_token}'
#             })
#     return(response.json())
        
        
#         url = f'https://api.spotify.com/v1/audio-features?ids={request_data}'
#         print(url)
#         response = requests.get(
#             url,
#             headers = {
#             'Accept': 'application/json',
#             'Content-Type': 'application/json',
#             'Authorization': f'Bearer {playlist_token}'
#             })

#         features_dict = response.json()
#         features_list = features_dict["audio_features"]
        
#         for i in features_list:
#             print(i)
            


    

In [5]:
import requests
import base64
import csv
import json
from datetime import datetime, date
from spotify_client import *
from config import *

def refresh_accesss_token():
    client_creds = f'{client_id}:{client_secret}'
    client_creds_b64 = base64.b64encode(client_creds.encode())

    refresh_token_header = {
        'Authorization' : f'Basic {client_creds_b64.decode()}'
    }
    # def refresh_playlist_token():
    refresh_url =  'https://accounts.spotify.com/api/token'
    refresh_params = {
        'grant_type': 'refresh_token',
        'refresh_token': 'AQA92cE5oL-fTQvG1_objGmNUVNQ_6AseJUqjsNC2ujDEFeMTOPxBpLqQ6Ct2tkmLIp79RT6deD3WjdPRYDlhnU2YgautBPNYw5aGMjYVD-y35lz5_n1L88yxG2UA-7nd58'        
    }
    refresh_data = urlencode(refresh_params)

    r_token = requests.post(refresh_url, data=refresh_params, headers=refresh_token_header)

    refresh_response = r_token.json()

    access_token = refresh_response['access_token']

    return access_token

track_ids = ['4c8smavpayWzKMgLze3m7w',
  '28kOGtTZzbfQ8fMmTwjRFq',
  '3grdgfHCm4mWVJliAitsE6',
  '7KBgmR0GBrUONdh0PNnyZD',
  '1qDXiACY4IrRsQ6mfTXrqA',
  '4ByvgSH9KPxMQqqXFR9797',
  '2M1QgxbjuF9GUDohSvdkN6',
  '1sXGfQ3TxrdAGeAoFGWGqD',
  '0nuTaK1uPw4BP4ql1n8ypr',
  '40lCE9TbjIe9e8ftO5JXkx',
  '2wNJEhEFe9IousfizPcbuQ',
  '6DK23fzeG32GDllJRfnDuz',
  '188Y8ny5w1GArwFjXE2arA',
  '0geFTYLpA8RJC6C8tftDiS',
  '0tO4X3FZeckjj7rRgBcjoK',
  '0DiidTJxEe6YaJG2H9fnZ8',
  '53V9TUAixZKpVplvjzuWmN',
  '1GrRRFtdrcLRc00p3DzcsZ',
  '10tPPpNb3zCIt2KKbWTXmx',
  '53HqOfi7OfHRXzcWTdGJB3',
  '53G34OGTpJUmiiTyQucr5F',
  '3BZAi7xzo0Kmpml96z1sJx',
  '2dxsMjCc2rceXeveoFAhCN',
  '75DmTnEYOy5At1DjnY9O6a',
  '7i6TSOhApYMJGKMpeMpl2T',
  '0pMt7yNoZTu1gQByxkXaMu',
  '1kNUlYIquh3WpllRoTCRJR',
  '5Xd9ChcgoAg1V0jxQ8pRc0',
  '1YhJTm664ZT5xheQLBDgd1',
  '29MtIa5NrOiDNPwIil7xzc',
  '2kU6bMVN21UbJ425q1yIKR',
  '2ocXJYrJavY7KTqIyTZNsD',
  '18mpSXpjDzwVTBSRDLKrTQ',
  '2g8y0qpGaM3HTWWJZiInKA',
  '4LbezHX4BdUnmogPcxiQCw',
  '0npTwyJYm09pWjuNPKlVbx',
  '7urL2WeDh4k81jFaWe3d8R',
  '379QAGtVDeZNXpr5ttTt6w',
  '3Jnsn0AUysVxzN9fgJIZPR',
  '7tLevkJnFLOMtOOUmhznQa',
  '77QKNMGwlnzxsTZPYfByMu',
  '5WGvXmLt6acluR7TfhlHUH',
  '107S1KOprUaRQqfI9nugxv',
  '3iZOVTPjyUEq9f5HsUQCj7',
  '6JGqh00P04EXR7pqkGQZIL',
  '1RkJ1vso9midZwy1FBIXAm',
  '0HPmWGyYJ0xP1DYUWk8Fwm',
  '6Y7GgtWd0aFBRtKtp0ZSsP',
  '3FZTvHvKC7v0oRoUXLxI9f',
  '6578mfa33CmqVLChEiIOFS',
  '6jfTZs0FvQ9iJhWGyNHSKI',
  '1gQma7UmtOdHu0UOsrnG7Q',
  '1mAEZUWQHb205V4n38azkB',
  '3ZMqMVi10xQVw5IXrsdfWs',
  '4ksPH6sLYv9EJ2bXbT8iQH',
  '05NosELMgN6lh7AApPzAXK',
  '1HLSQYyZ8g5UXwrC8q7tSL',
  '63o8UQjqO4lGp36L0sTcDB',
  '2F2Nhhhjzs4zLoWle1AqN1',
  '4GI09BcZE3zITBpxLXFRFM',
  '6j8FNLYW8epgHtcgZFmTFq',
  '4p4qSBvMGa7BQrtSKdLNGq',
  '0zlaa4AmbFE7MVDijsyD9Q',
  '32NJ4kh1Aqrc9XGRoXvGie',
  '2lXGcshKbu1QSq04zEIjAD',
  '3qaRqV7dTbQV2VVMhd0ejd',
  '6XALorRRo9aQdBIQN0UWyn',
  '2TYaHbldGeJFXQkMY1hbwC',
  '07p7PALHp6ZcD5tmlbO94N',
  '4iya38ctQ1httyA6NOG9oW',
  '0ArFo2RpDxYMMZzo2nr6yh',
  '3RQjHiFJnKQKFKjQ7XI8DD',
  '6ml28M2pzhbhWAv0cp6Nfy',
  '6cFrujlrXPM4mm9ZCce50F',
  '5wIea6rmDFbFQAlLnhEv8a',
  '2K2b3lVP4NcKmrJKEq96iU',
  '1cBXnJrodwBz89ylkEpKHJ',
  '15eY7ByxPYQKpF3FSnXadX',
  '0vcvxYWADFt9v9ZzdZ606N',
  '1kxob0ar9s23FbJKFTakOK',
  '4evRqQUexMROsP0fyIuYgV',
  '66KaEPftgdj3MvwOK1Obfn',
  '2POFtKZA1lsXksmDf78Ozg',
  '1MUMeTbRDcMz4GDbZILGcU',
  '3B7zM6Ebe8udFOmpLNYYpO',
  '2BoWMwR1iCy4OHYbwkMuGX',
  '0krpBelJJZNBQvAr3yPL3X',
  '5Nt62jCYyNKtllWgA9DtAC',
  '3eGmwXNubL1bLaUiA4Qj8O',
  '6IOL5tW3yRKKKpPNVCVmzU',
  '35v95h9sBcsb9P0rmHXd6U',
  '5SabDEPhACfKFiMG5HaJTR',
  '56ToPbtCmQrWibtlu6Sa1T',
  '07gNCLMv4WLXtntHi2Pa2o',
  '1vkLJondcibYblvTt3dfec',
  '6XvM8pdQvqbJe7kPCHrCyt',
  '2tNsZxAdz89tXBLo620O5E',
  '7wwCmOY6H2PIOCjbHpzF6y',
  '31Kn2JycP5A1pa6tPnLQKP',
  '6sKSIPIAeIyiHkFkI69qOW']

def get_track_features(track_ids):
    track_ids = [track for track in track_ids]
    track_limit = 100 
    playlist_token = refresh_accesss_token()
    # using list comprehension 
    batched_tracks = [track_ids[i * track_limit:(i + 1) * track_limit] for i in range((len(track_ids) + track_limit - 1) // track_limit)]  
    for batch in batched_tracks:
        request_data = json.dumps(batch)
        params= request_data
        
#   search for album ids 

        url = 'https://api.spotify.com/v1/audio-features?id='
        
        response = requests.get(
            url,
            params=params,
            headers = {
            'Accept': 'application/json',
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {playlist_token}'
            })
        a = response.json()
        print(a)


In [6]:
get_track_features(track_ids)

{'audio_features': []}


In [257]:
import requests
import base64
import csv
import json
from datetime import datetime, date
from spotify_client import *
from config import *

def get_client_credentials(self):
    """
    Returns a base64 encoded string
    """
    client_id = self.client_id
    client_secret = self.client_secret
    if client_secret == None or client_id == None:
        raise Exception("You must set client_id and client_secret")
    client_creds = f"{client_id}:{client_secret}"
    client_creds_b64 = base64.b64encode(client_creds.encode())
    return client_creds_b64.decode()

def refresh_accesss_token():
    client_creds = f'{client_id}:{client_secret}'
    client_creds_b64 = base64.b64encode(client_creds.encode())
    
    refresh_token_header = {
        'Authorization' : f'Basic {client_creds_b64.decode()}'
    }

    # def refresh_playlist_token():
    refresh_url =  'https://accounts.spotify.com/api/token'
    refresh_params = {
        'grant_type': 'refresh_token',
        'refresh_token': 'AQA92cE5oL-fTQvG1_objGmNUVNQ_6AseJUqjsNC2ujDEFeMTOPxBpLqQ6Ct2tkmLIp79RT6deD3WjdPRYDlhnU2YgautBPNYw5aGMjYVD-y35lz5_n1L88yxG2UA-7nd58'
        
    }

    refresh_data = urlencode(refresh_params)



    r_token = requests.post(refresh_url, data=refresh_params, headers=refresh_token_header)

    refresh_response = r_token.json()

    access_token = refresh_response['access_token']

    return access_token


# def get_week_num():
playlist_token = refresh_accesss_token()
csv_path = path
def get_week_num():
    my_date = datetime.date.today() 
    year, week_num, day_of_week = my_date.isocalendar()
    this_week = week_num
    return this_week


def get_album_tracks(album_ids):
    track_ids = []
    track_uris = []
#     request_data = json.dumps(uris)
    for album_id in album_ids:
        query = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
        response = requests.get(
            query,
            headers = {
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {playlist_token}'
            })
        response_json = response.json()
#         print(response_json)
        tracks = response_json['items']
        for track in tracks:
            track_ids.append(track['id'])
            track_uris.append(track['uri'])
    print(len(track_ids))
    get_track_features(track_ids)    
    return track_uris

def search_for_albums(csv_path):
    artists=[]
    albums=[]
    albums_not_found = {'artist': [], 'album': []}
    album_ids = set()

#     read in weekly metaScrape csv 
    with open(csv_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
#         filter for artists and albums from current week
            if int(row['week_num'])==15:
                artists.append(row['artist'])
                albums.append(row['album'])
#   initialize spotify client
    spotify = SpotifyAPI(client_id, client_secret)
    for al, ar in zip(albums, artists):
#   search for album ids 
        temp = spotify.search({"album":al, "artist":ar}, search_type="album")
        try:
            parse_album_ids = (temp["albums"]["items"][0]["id"])
#   create dicitonary for albums not found
        except:
            albums_not_found['artist'].append(ar)
            albums_not_found['album'].append(al)
        album_ids.add(parse_album_ids)
    return(get_album_tracks(album_ids))
'''
 response body contains an object whose key is "audio_features" and whose value is an array of 
 audio features objects in JSON format.
 
Objects are returned in the order requested. If an object is not found, a null value is returned in the appropriate position. 
Duplicate ids in the query will result in duplicate objects in the response. 
On error, the header status code is an error code and the response body contains an error object.

'''
def get_track_features(track_ids):
    track_ids = [track for track in track_ids]
    track_limit = 100 
    # using list comprehension 
    batched_tracks = [track_ids[i * track_limit:(i + 1) * track_limit] for i in range((len(track_ids) + track_limit - 1) // track_limit)]  
    for batch in batched_tracks:
        request_data = json.dumps(batch)
        params= request_data
        
#   search for album ids 

        url = 'https://api.spotify.com/v1/audio-features'
        
        response = requests.get(
            url,
            params=params,
            headers = {
            'Accept': 'application/json',
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {playlist_token}'
            })
        a = response.json()
        print(a.keys())
       
#  def base_search(self, query_params):
#      headers = self.get_resource_header()
#      endpoint = "https://api.spotify.com/v1/search"
#      lookup_url = f"{endpoint}?{query_params}"
#      r = requests.get(lookup_url, headers=headers)
#      if r.status_code not in range(200, 299):
#          return {}
#      return r.json()       
    

def add_tracks_to_playlist(playlist_id):
#     get_track_features()
    track_uris = [track for track in search_for_albums(csv_path)]
    track_limit = 100 
    # using list comprehension 
    batched_tracks = [track_uris[i * track_limit:(i + 1) * track_limit] 
                      for i in range((len(track_uris) + track_limit - 1) // track_limit)]  
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'

    for batch in batched_tracks:
        
        request_data = json.dumps(batch)
        response = requests.post(
            url,
            data=request_data,
            headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {playlist_token}'
            })
    return(response.json())

def create_playlist():
    week_num = get_week_num()
    request_body = json.dumps({
        'name': f'2021-week {week_num} scrape',
        'description': f'metacritic rated albums for the {week_num}th of the year',
        'public': True
    })
    url = f'https://api.spotify.com/v1/users/{spotify_user_id}/playlists'


    response = requests.post(
        url,
        data = request_body,
        headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {playlist_token}'

    })
    response_json = response.json()
#     print(response_json)
    playlist_id = (response_json['id'])

    return add_tracks_to_playlist(playlist_id)

In [258]:
create_playlist()

100
['4c8smavpayWzKMgLze3m7w', '28kOGtTZzbfQ8fMmTwjRFq', '3grdgfHCm4mWVJliAitsE6', '7KBgmR0GBrUONdh0PNnyZD', '1qDXiACY4IrRsQ6mfTXrqA', '4ByvgSH9KPxMQqqXFR9797', '2M1QgxbjuF9GUDohSvdkN6', '1sXGfQ3TxrdAGeAoFGWGqD', '0nuTaK1uPw4BP4ql1n8ypr', '40lCE9TbjIe9e8ftO5JXkx', '2wNJEhEFe9IousfizPcbuQ', '6DK23fzeG32GDllJRfnDuz', '188Y8ny5w1GArwFjXE2arA', '0geFTYLpA8RJC6C8tftDiS', '0tO4X3FZeckjj7rRgBcjoK', '0DiidTJxEe6YaJG2H9fnZ8', '53V9TUAixZKpVplvjzuWmN', '1GrRRFtdrcLRc00p3DzcsZ', '10tPPpNb3zCIt2KKbWTXmx', '53HqOfi7OfHRXzcWTdGJB3', '53G34OGTpJUmiiTyQucr5F', '3BZAi7xzo0Kmpml96z1sJx', '2dxsMjCc2rceXeveoFAhCN', '75DmTnEYOy5At1DjnY9O6a', '7i6TSOhApYMJGKMpeMpl2T', '0pMt7yNoZTu1gQByxkXaMu', '1kNUlYIquh3WpllRoTCRJR', '5Xd9ChcgoAg1V0jxQ8pRc0', '1YhJTm664ZT5xheQLBDgd1', '29MtIa5NrOiDNPwIil7xzc', '2kU6bMVN21UbJ425q1yIKR', '2ocXJYrJavY7KTqIyTZNsD', '18mpSXpjDzwVTBSRDLKrTQ', '2g8y0qpGaM3HTWWJZiInKA', '4LbezHX4BdUnmogPcxiQCw', '0npTwyJYm09pWjuNPKlVbx', '7urL2WeDh4k81jFaWe3d8R', '379QAGtVDeZNXpr5ttTt6w', '3Jnsn0

{'snapshot_id': 'MyxhZmNiZmVjYzc3MmQ5ZWUxZjQ0NDIzNTU4NjdlYTJhYTBkN2NkYTk1'}